In [50]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading the dataset & Pre trained word embeddings

In [51]:
import os
import torch
import torchtext.data
import torchtext.datasets
import numpy as np 
import time

from RNN import SentimentGRU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device =", device)

device = cuda


load the data set:

In [52]:

data_dir = os.path.expanduser('~/.pytorch-datasets')

# torchtext Field objects parse text (e.g. a review) and create a tensor representation

# This Field object will be used for tokenizing the movie reviews text
review_parser = torchtext.data.Field(
    sequential=True, use_vocab=True, lower=True,
    init_token='<sos>', eos_token='<eos>', dtype=torch.long,
    tokenize='spacy', tokenizer_language='en_core_web_sm'
)

# This Field object converts the text labels into numeric values (0,1,2)
label_parser = torchtext.data.Field(
    is_target=True, sequential=False, unk_token=None, use_vocab=True
)

# Load SST, tokenize the samples and labels
# ds_X are Dataset objects which will use the parsers to return tensors
ds_train, ds_valid, ds_test = torchtext.datasets.SST.splits(
    review_parser, label_parser, root=data_dir
)

n_train = len(ds_train)
print(f'Number of training   samples: {n_train}')
print(f'Number of validation samples: {len(ds_valid)}')
print(f'Number of test       samples: {len(ds_test)}')

Number of training   samples: 8544
Number of validation samples: 1101
Number of test       samples: 2210


As required, we'll use the pre-trained word embeddings of glove 6B.

In [53]:
#Vocabulary size is 40k, Embedding chosen size in 50
vocab, embeddings = [],[]
with open('./GloVe/glove.6B.50d.txt','rt',encoding='utf8') as fi:
    full_content = fi.read().strip().split('\n')
for i in range(len(full_content)):
    i_word = full_content[i].split(' ')[0]
    i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
    vocab.append(i_word)
    embeddings.append(i_embeddings)
    

add padding and unknown tokens to the embeddings array:

In [54]:
# Add the padding and the unknown tokens to the vocab and embeddings arrays

vocab = np.array(vocab) 
embeddings = np.array(embeddings)
vocab = np.insert(vocab, 0, '<pad>')
vocab = np.insert(vocab, 1, '<unk>')

unk_emb = np.mean(embeddings, axis=0, keepdims=True)
pad_emb = np.zeros_like(embeddings[0]).reshape(1,-1)


embeddings = np.vstack((pad_emb, unk_emb, embeddings))

print(vocab[:10])

['<pad>' '<unk>' 'the' ',' '.' 'of' 'to' 'and' 'in' 'a']


## Baseline Model - Sentiment Analysis using RNN - GRU

As for the first part in our experiment




In [55]:
def train(model, optimizer, loss_fn, dataloader, max_epochs=100,
          num_batches=400, save_path=None):
    train_losses = []
    train_acc = []
    
    for epoch_idx in range(max_epochs):
        total_loss, num_correct = 0, 0
        start_time = time.time()

        for batch_idx, batch in enumerate(dataloader):
            X, y = batch.text, batch.label

            # Forward pass
            _, y_pred_log_proba = model(X)

            # Backward pass
            optimizer.zero_grad()
            loss = loss_fn(y_pred_log_proba, y)
            loss.backward()

            # Weight updates
            optimizer.step()

            # Calculate accuracy
            total_loss += loss.item()
            y_pred = torch.argmax(y_pred_log_proba, dim=1)
            num_correct += torch.sum(y_pred == y).float().item()

            if batch_idx == num_batches-1:
                break
        
        curr_train_loss = total_loss /(num_batches)
        curr_train_acc = num_correct /(num_batches*BATCH_SIZE)
        train_losses.append(curr_train_loss)
        train_acc.append(curr_train_acc)
        
        
        print(f"Epoch #{epoch_idx}, loss={curr_train_loss:.3f}, accuracy={curr_train_acc:.3f}, elapsed={time.time()-start_time:.1f} sec")
        
        if save_path:
            torch.save(model, save_path)
            
    return train_losses, train_acc

In [56]:
EMBEDDING_DIM = 50
BATCH_SIZE = 32
HIDDEN_SIZE = 256
NUM_LAYERS = 2
DROPOUT = 0
LEARNING_RATE = 1e-3

In [57]:
dl_train, dl_valid, dl_test = torchtext.data.BucketIterator.splits(
    (ds_train, ds_valid, ds_test), batch_size=BATCH_SIZE,
    shuffle=True, device=device)
review_parser.build_vocab(ds_train)
label_parser.build_vocab(ds_train)

In [58]:

model = SentimentGRU(embeddings, hidden_size=HIDDEN_SIZE,
                     num_layers=NUM_LAYERS, dropout=DROPOUT).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

loss_fn = torch.nn.NLLLoss()


In [59]:
model = torch.load("./models/sentimentGRU.pt")
print(model)

SentimentGRU(
  (embedding_layer): Embedding(400002, 50)
  (gru): GRU(50, 256, num_layers=2)
  (dense_linear): Linear(in_features=256, out_features=3, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)


In [14]:
train_losses, train_accs = train(model, optimizer, loss_fn, dl_train, max_epochs=100,
      num_batches=500, save_path = "./models/sentimentGRU.pt")

C:\Users\dorbi\miniconda3\envs\cs236781-hw\lib\site-packages\torchtext\data\batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


tensor(1.0951, grad_fn=<NllLossBackward>)
tensor(1.0712, grad_fn=<NllLossBackward>)
tensor(0.9919, grad_fn=<NllLossBackward>)
tensor(1.0121, grad_fn=<NllLossBackward>)
tensor(1.0792, grad_fn=<NllLossBackward>)
tensor(1.0924, grad_fn=<NllLossBackward>)
tensor(1.0322, grad_fn=<NllLossBackward>)
tensor(1.1190, grad_fn=<NllLossBackward>)
tensor(1.1356, grad_fn=<NllLossBackward>)
tensor(1.0257, grad_fn=<NllLossBackward>)
tensor(1.0392, grad_fn=<NllLossBackward>)
tensor(1.0709, grad_fn=<NllLossBackward>)
tensor(1.1263, grad_fn=<NllLossBackward>)


KeyboardInterrupt: 

And evaluate on the test set:


In [60]:
def test_epoch(model, loss_fn, dataloader)
    model.eval()

    test_loss = 0
    num_correct = 0 
    num_batches = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            num_batches = batch_idx
            X, y = batch.text, batch.label

            _, y_test = model(X)
            loss = loss_fn(y_test, y)

            # Calculate accuracy
            test_loss += loss.item()
            y_pred = torch.argmax(y_test, dim=1)
            num_correct += torch.sum(y_pred == y).float().item()
            
                 
        test_loss = total_loss /(num_batches)
        test_acc = num_correct /(num_batches*BATCH_SIZE)

    model.train()
    return test_loss, test_acc

SyntaxError: invalid syntax (<ipython-input-60-57c53c1059b7>, line 1)

In [69]:
from SelfAttention import SentimentSelfAttention

model_attention = SentimentSelfAttention(embeddings, num_heads=1).to(device)
print(model_attention)

print("trainable params:", 
      sum(p.numel() for p in model_attention.parameters() if p.requires_grad)) 



SentimentSelfAttention(
  (embedding_layer): Embedding(400002, 50)
  (q_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
  )
  (k_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
  )
  (v_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
  )
  (PositionalEncoding): PositionalEncoding()
  (SelfAttention1): MultiheadAttention(
    (out_proj): _LinearWithBias(in_features=50, out_features=50, bias=True)
  )
  (dense_linear): Linear(in_features=50, out_features=3, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)
trainable params: 18003


In [70]:
batch = next(iter(dl_train)).text.to(device)
print("batch shape:", batch.shape)
_ = model_attention(batch)

batch shape: torch.Size([49, 32])


In [71]:
train(model_attention, optimizer, loss_fn, dl_train, max_epochs=100,
          num_batches=400, save_path=None)

Epoch #0, loss=0.742, accuracy=0.282, elapsed=0.9 sec
Epoch #1, loss=0.742, accuracy=0.282, elapsed=0.9 sec
Epoch #2, loss=0.743, accuracy=0.282, elapsed=0.9 sec
Epoch #3, loss=0.742, accuracy=0.282, elapsed=0.9 sec
Epoch #4, loss=0.742, accuracy=0.282, elapsed=0.9 sec
Epoch #5, loss=0.742, accuracy=0.282, elapsed=0.9 sec
Epoch #6, loss=0.743, accuracy=0.282, elapsed=0.9 sec
Epoch #7, loss=0.743, accuracy=0.282, elapsed=0.9 sec
Epoch #8, loss=0.742, accuracy=0.282, elapsed=0.9 sec
Epoch #9, loss=0.743, accuracy=0.282, elapsed=0.9 sec
Epoch #10, loss=0.742, accuracy=0.282, elapsed=0.9 sec
Epoch #11, loss=0.742, accuracy=0.282, elapsed=1.0 sec
Epoch #12, loss=0.743, accuracy=0.282, elapsed=1.0 sec
Epoch #13, loss=0.742, accuracy=0.282, elapsed=1.0 sec
Epoch #14, loss=0.742, accuracy=0.282, elapsed=1.0 sec
Epoch #15, loss=0.742, accuracy=0.282, elapsed=1.1 sec


KeyboardInterrupt: 